## Импорт

In [8]:
import requests
from user_agent import generate_user_agent
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType, StringType
from pyspark.sql.functions import sum, map_zip_with, round, col, max, min, stddev, count

## Часть 1

In [4]:
HEADERS = {'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))}
URL = 'http://notelections.online'
LINK = 'http://notelections.online/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227'

In [6]:
response = requests.get(LINK, timeout=5, headers=HEADERS)
df_columns = ['Регион', 'ТИК', 'УИК']

if response.status_code == 200:
    bs = BeautifulSoup(response.text, 'html.parser')
    
    li_table = bs.find('table', class_='table-fixed-columns')
    li_list = li_table.findAll('th')[3:]

    columns = li_table.find('tbody').findAll('tr')
    
    for column in columns:
        df_columns.append(column.findAll('td')[1].text)

    df = pd.DataFrame(columns=df_columns)

    for li in tqdm(li_list[:-2]):
        link = li.find('a').attrs['href']
        region_response = requests.get(URL + link, timeout=10, headers=HEADERS)

        if region_response.status_code == 200:
            region_bs = BeautifulSoup(region_response.text, 'html.parser')

            li_list_2 = region_bs.find('table', class_='table-fixed-columns').findAll('th')[3:]

            for li_2 in li_list_2:
                link_tik = li_2.find('a')
                href = link_tik.attrs['href']

                tik_response = requests.get(URL + href, timeout=10, headers=HEADERS)

                if tik_response.status_code == 200:
                    tik_bs = BeautifulSoup(tik_response.text, 'html.parser')

                    table = tik_bs.find('table', class_='table-fixed-columns')

                    # список всех уиков:
                    head = table.findAll('th')[2:]
                    col_count = len(head)

                    rows = table.find('tbody').findAll('tr')
                    row_count = len(rows)

                    data = [[] * row_count for i in range(col_count)]

                    for row in rows:
                        td = row.findAll('td')[2:]
                        
                        for t in range(len(td)):
                            text = td[t].find('nobr').text
                            data[t].append(text)


                    for data_row in data:
                        append_row = pd.Series([li.text, link_tik.text, head[data.index(data_row)].text, data_row[0], data_row[1], data_row[2], data_row[3], data_row[4], data_row[5], data_row[6], data_row[7], data_row[8], data_row[9], data_row[10], data_row[11], data_row[12], data_row[13], data_row[14], data_row[15], data_row[16], data_row[17], data_row[18], data_row[19]], index=df.columns)
                        df = df.append(append_row, ignore_index=True)

df_add = pd.DataFrame(columns=df_columns)

for li in tqdm(li_list[85:]):
    link = li.find('a').attrs['href']
    region_response = requests.get(URL + link, timeout=10, headers=HEADERS)

    if region_response.status_code == 200:
        region_bs = BeautifulSoup(region_response.text, 'html.parser')

        table = region_bs.find('table', class_='table-fixed-columns')

        # список всех уиков:
        head = table.findAll('th')[2:]
        col_count = len(head)

        rows = table.find('tbody').findAll('tr')
        row_count = len(rows)

        data = [[] * row_count for i in range(col_count)]

        for row in rows:
            td = row.findAll('td')[2:]
            
            for t in range(len(td)):
                text = td[t].find('nobr').text
                data[t].append(text)


        for data_row in data:
            append_row = pd.Series([li.text, None, head[data.index(data_row)].text, data_row[0], data_row[1], data_row[2], data_row[3], data_row[4], data_row[5], data_row[6], data_row[7], data_row[8], data_row[9], data_row[10], data_row[11], data_row[12], data_row[13], data_row[14], data_row[15], data_row[16], data_row[17], data_row[18], data_row[19]], index=df_add.columns)
            df_add = df_add.append(append_row, ignore_index=True)
            
df = df.append(df_add)
df.to_csv('elections_data.csv')

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.80s/it]


In [9]:
df

,Регион,ТИК,УИК,"Число избирателей, включенных в список избирателей","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно","Число избирательных бюллетеней, выданных в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования",Число погашенных избирательных бюллетеней,Число избирательных бюллетеней в переносных ящиках для голосования,...,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",Бабурин Сергей Николаевич,Грудинин Павел Николаевич,Жириновский Владимир Вольфович,Путин Владимир Владимирович,Собчак Ксения Анатольевна,Сурайкин Максим Александрович,Титов Борис Юрьевич,Явлинский Григорий Алексеевич
0,Республика Адыгея (Адыгея),Адыгейская,Сумма,11624,11350,0,10833,399,118,399,...,0,0,24,655,149,10229,62,16,15,25
1,Республика Адыгея (Адыгея),Адыгейская,УИК №1,2256,2181,0,2107,62,12,62,...,0,0,0,137,32,1977,14,0,1,5
2,Республика Адыгея (Адыгея),Адыгейская,УИК №2,2700,2633,0,2575,41,17,41,...,0,0,15,86,65,2389,13,5,6,15
3,Республика Адыгея (Адыгея),Адыгейская,УИК №3,2858,2752,0,2664,75,13,75,...,0,0,1,62,13,2645,6,3,4,0
4,Республика Адыгея (Адыгея),Адыгейская,УИК №4,2066,2034,0,1857,142,35,142,...,0,0,5,288,12,1642,21,6,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,Территория за пределами РФ,None,УИК №8433,436,2500,0,431,5,2064,5,...,0,0,3,32,13,371,8,3,3,3
398,Территория за пределами РФ,None,УИК №8434,1141,3000,0,1136,5,1859,5,...,0,0,1,10,5,1072,8,1,0,0
399,Территория за пределами РФ,None,УИК №8436,78,1507,0,78,0,1429,0,...,0,0,0,17,3,43,10,0,3,2
400,Территория за пределами РФ,None,УИК №8437,848,2500,0,743,105,1652,105,...,0,0,4,4,0,802,1,1,0,0


## Часть 2

In [12]:
spark = SparkSession.builder.master("local[1]").appName("elections").getOrCreate()

schema = StructType()\
    .add("id",IntegerType(),False)\
    .add("region",StringType(),False)\
    .add("tik",StringType(),False)\
    .add("uik",StringType(),False)\
    .add("includedVoters",IntegerType(),False)\
    .add("recievedBallots",IntegerType(),False)\
    .add("earlyBallots",IntegerType(),False)\
    .add("stationBallots",IntegerType(),False)\
    .add("outStationBallots",IntegerType(),False)\
    .add("canceledBallots",IntegerType(),False)\
    .add("portableBallots",IntegerType(),False)\
    .add("stationaryBallots",IntegerType(),False)\
    .add("invalidBallots",IntegerType(),False)\
    .add("validBallots",IntegerType(),False)\
    .add("lostBallots",IntegerType(),False)\
    .add("notRecordedBallots",IntegerType(),False)\
    .add("Baburin",IntegerType(),False)\
    .add("Grudinin",IntegerType(),False)\
    .add("Zhirinovsky",IntegerType(),False)\
    .add("Putin",IntegerType(),False)\
    .add("Sobchak",IntegerType(),False)\
    .add("Suraykin",IntegerType(),False)\
    .add("Titov",IntegerType(),False)\
    .add("Yavlinsky",IntegerType(),False)

df = spark.read.format('csv').option("header",True).schema(schema).load("elections_data.csv")

## Найти явку (%) по всем регионам, результат отсортировать по убыванию

In [13]:
data = df.select('region','validBallots', 'includedVoters', 'uik') \
    .filter('uik=="Сумма"') \
    .groupBy('region') \
    .agg(sum('validBallots').alias('validBallots_sum'),sum('includedVoters').alias('includedVoters_sum')) \
    .withColumn('percent', round(col('validBallots_sum') / col('includedVoters_sum') * 100, 2)) \
    .sort(col('percent').desc()) \
    .select('region','percent') \
    .show()

+--------------------+-------+
|              region|percent|
+--------------------+-------+
|Территория за пре...|  96.82|
|     Республика Тыва|  92.96|
|Кабардино-Балкарс...|  91.58|
|Ямало-Ненецкий ав...|  91.17|
|Чеченская Республика|  91.16|
|Республика Северн...|  89.26|
| Республика Дагестан|  87.06|
|Карачаево-Черкесс...|  86.97|
| Кемеровская область|  82.54|
|Республика Ингушетия|  81.64|
|Чукотский автоном...|  81.39|
|    Брянская область|  78.93|
|   Тюменская область|  78.21|
| Республика Мордовия|  77.26|
|  Краснодарский край|  77.19|
|Республика Татарс...|  76.75|
|Чувашская Республ...|  75.14|
|Республика Башкор...|  74.81|
|  Республика Бурятия|   74.2|
|Республика Адыгея...|  73.52|
+--------------------+-------+
only showing top 20 rows



## Выбрать любимого кандидата и найти тот избиратльный участок, на котором он получил наибольший результат

In [14]:
waifu = 'Grudinin'

print(waifu + " results:\n")

df.select('region','tik', 'uik', waifu, 'validBallots') \
    .filter('uik!="Сумма"') \
    .filter('validBallots > 300') \
    .withColumn('percent', round((col(waifu) / col('validBallots')) * 100, 2)) \
    .sort(col('percent').desc()) \
    .select('region','tik', 'uik', 'percent') \
    .show(1)

Grudinin results:

+------------------+---------+---------+-------+
|            region|      tik|      uik|percent|
+------------------+---------+---------+-------+
|Московская область|Ленинская|УИК №1306|   67.0|
+------------------+---------+---------+-------+
only showing top 1 row



## Найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна

In [15]:
data = df.select('region','validBallots', 'includedVoters', 'uik') \
    .filter('uik=="Сумма"') \
    .withColumn('percent', round((col('validBallots')) / col('includedVoters') * 100, 2)) \
    .groupBy('region') \
    .agg(max('percent').alias('percent_max'),min('percent').alias('percent_min')) \
    .withColumn('difference', round(col('percent_max') - col('percent_min'), 2)) \
    .sort(col('difference').desc()) \
    .select('region', 'difference') \
    .show(1)

+--------------------+----------+
|              region|difference|
+--------------------+----------+
|Архангельская обл...|     49.58|
+--------------------+----------+
only showing top 1 row



## Посчитать дисперсию по явке для каждого региона (учитывать УИК)

In [16]:
df.select('region','validBallots', 'includedVoters', 'uik', round(col('validBallots') / col('includedVoters'), 2).alias('percent')) \
    .filter('uik=="Сумма"') \
    .groupBy('region') \
    .agg(stddev(col('percent')).alias('dispersion')) \
    .show()

+--------------------+--------------------+
|              region|          dispersion|
+--------------------+--------------------+
|Республика Саха (...| 0.07522291801677189|
|Калининградская о...| 0.07755316082290385|
|Новосибирская обл...| 0.04558486369377229|
|   город Севастополь|0.015000000000000024|
|Свердловская область| 0.04508414963185734|
|  Республика Хакасия| 0.04778423728943818|
| Магаданская область| 0.07983315935958774|
|Республика Ингушетия| 0.05833333333333332|
|Удмуртская Респуб...| 0.05058393423659384|
|     Камчатский край| 0.11266046756684864|
| Саратовская область|   0.096450415764799|
|Республика Башкор...|  0.0819607333259752|
|  Ростовская область| 0.08018157606419005|
|   Калужская область| 0.07031580312641085|
|Еврейская автоном...|                0.04|
|Карачаево-Черкесс...| 0.07149380138420142|
| Республика Дагестан| 0.07615681346797598|
|Ханты-Мансийский ...| 0.08015137194235934|
|Республика Татарс...| 0.11640075006366844|
|Ямало-Ненецкий ав...|0.04771711

## Для каждого кандидата посчитать таблицу: результат (%, округленный до целого) - количество УИК, на которых кандидат получил данный результат

In [17]:
for person in ['Baburin', 'Grudinin', 'Zhirinovsky', 'Putin', 'Sobchak', 'Suraykin', 'Titov', 'Yavlinsky']:
    print(person + " results:\n")
    df.select('region','tik', 'uik', person, 'validBallots') \
        .filter('uik!="Сумма"') \
        .withColumn('percent', round((col(person) / col('validBallots')) * 100)) \
        .groupBy('percent') \
        .agg(count('uik').alias('uik_count')) \
        .sort(col('percent').desc()) \
        .show()

Baburin results:

+-------+---------+
|percent|uik_count|
+-------+---------+
|   44.0|        1|
|   23.0|        1|
|   21.0|        2|
|   20.0|        2|
|   18.0|        2|
|   17.0|        1|
|   15.0|        2|
|   14.0|        2|
|   13.0|        1|
|   12.0|        1|
|   11.0|        6|
|   10.0|        9|
|    9.0|       10|
|    8.0|       21|
|    7.0|       21|
|    6.0|       38|
|    5.0|       86|
|    4.0|      186|
|    3.0|      586|
|    2.0|     3918|
+-------+---------+
only showing top 20 rows

Grudinin results:

+-------+---------+
|percent|uik_count|
+-------+---------+
|   80.0|        1|
|   78.0|        1|
|   75.0|        1|
|   71.0|        1|
|   70.0|        1|
|   69.0|        2|
|   67.0|        4|
|   61.0|        1|
|   60.0|        1|
|   58.0|        3|
|   57.0|        6|
|   56.0|        4|
|   55.0|        3|
|   54.0|        5|
|   53.0|        4|
|   52.0|        4|
|   51.0|        4|
|   50.0|       12|
|   49.0|        5|
|   48.0|        